In [8]:
import numpy as np

# Lendo o arquivo de entrada e preparando a tabela inicial

In [12]:
def ler_entrada(arquivo):
    try:
        with open(arquivo, 'r') as f:
            linhas = [linha.strip() for linha in f.readlines()]
        
        if len(linhas) < 5:
            raise ValueError("O arquivo deve ter pelo menos 5 linhas: número de origens, número de destinos, capacidades de atendimento de cada origem, demandas de cada destino, e matriz de custos de transporte.")
        
        try:
            num_origens = int(linhas[0])
            if num_origens <= 0:
                raise ValueError("O número de origens deve ser um inteiro positivo.")
        except ValueError:
            raise ValueError("A primeira linha deve ser um número inteiro positivo.")

        try:
            num_destinos = int(linhas[1])
            if num_destinos <= 0:
                raise ValueError("O número de destinos deve ser um inteiro positivo.")
        except ValueError:
            raise ValueError("A segunda linha deve ser um número inteiro positivo indicando o número de destinos.")
        
        try:
            cap_atendimento = linhas[2].replace(', ', ' ').split()
            cap_atendimento = [int(x) for x in cap_atendimento]
            if len(cap_atendimento) != num_origens:
                print(f"O número de origens é {num_origens}.")
                print(f"O número de cap é {len(cap_atendimento)}.")
                raise ValueError("A capacidade de atendimento deve ser um vetor com pelo menos 1 posição.")
        except ValueError:
            raise ValueError("A terceira linha deve ser um vetor com pelo menos 1 posição.")
        
        try:
            demandas = linhas[3].replace(', ', ' ').split()
            demandas = [int(x) for x in demandas]
            if len(demandas) != num_destinos:
                print(f"O número de demandas é {len(demandas)}.")
                print(f"O número de destinos é {num_destinos}.")
                raise ValueError(f"A quantidade de demandas deve ser igual ao número de destinos ({num_destinos}).")
        except ValueError:
            raise ValueError("A quarta linha deve ser um vetor com pelo menos 1 posição.")

        if len(linhas) != 4 + num_origens:
            raise ValueError(f"O arquivo deve ter exatamente {4 + num_origens} linhas, mas possui {len(linhas)}.")
        
        # Lendo a matriz de custos
        try:
            matriz_custos = np.zeros((num_origens, num_destinos), dtype=np.float64)
            for i in range(num_origens):
                valores = linhas[4 + i].replace(', ', ' ').split()
                if len(valores) != num_destinos:
                    raise ValueError(f"A linha {5 + i} deve conter exatamente {num_destinos} valores.")
                matriz_custos[i, :] = [float(x) for x in valores]
        except ValueError:
            raise ValueError("A quarta linha deve conter os coeficientes da função objetivo, separados por vírgulas ou espaços.")
        
        return num_origens, num_destinos, cap_atendimento, demandas, matriz_custos

    except FileNotFoundError:
        raise FileNotFoundError(f"O arquivo '{arquivo}' não foi encontrado.")
    except Exception as e:
        raise RuntimeError(f"Erro ao ler o arquivo: {e}")

In [13]:
# Teste de leitura e preparação da tabela
arquivo = 'Entrada Problema de Transporte.txt'
num_origens, num_destinos, cap_atendimento, demandas, matriz_custos = ler_entrada(arquivo)

print(f"Número de origens: {num_origens}")
print(f"Número de destinos: {num_destinos}")
print(f"Demandas: {demandas}")
print(f"Capacidades de atendimento: {cap_atendimento}")
print("Matriz de custos:")
print(matriz_custos)

Número de origens: 3
Número de destinos: 2
Demandas: [2300, 1400]
Capacidades de atendimento: [1000, 1500, 1200]
Matriz de custos:
[[ 80. 215.]
 [100. 108.]
 [102.  68.]]


In [14]:
import numpy as np

def metodo_menor_custo(num_origens, num_destinos, cap_atendimento, demandas, matriz_custos):
    alocacao = np.zeros((num_origens, num_destinos), dtype=np.float64)
    
    # Convertendo listas para arrays numpy para facilitar operações
    capacidade = np.array(cap_atendimento, dtype=np.float64)
    demanda = np.array(demandas, dtype=np.float64)
    custos = np.array(matriz_custos, dtype=np.float64)
    
    # Criando cópia das capacidades e demandas para atualizar conforme alocação
    capacidade_restante = capacidade.copy()
    demanda_restante = demanda.copy()
    
    while np.any(capacidade_restante > 0) and np.any(demanda_restante > 0):
        # Encontrando a célula de menor custo
        custo_min = np.inf
        pos_min = (-1, -1)
        
        for i in range(num_origens):
            for j in range(num_destinos):
                if capacidade_restante[i] > 0 and demanda_restante[j] > 0:
                    if custos[i, j] < custo_min:
                        custo_min = custos[i, j]
                        pos_min = (i, j)
        
        i, j = pos_min
        
        # Determinar a quantidade a ser alocada na célula de menor custo
        quantidade = min(capacidade_restante[i], demanda_restante[j])
        alocacao[i, j] = quantidade
        
        # Atualizar capacidade e demanda restantes
        capacidade_restante[i] -= quantidade
        demanda_restante[j] -= quantidade
    
    return alocacao

# Exemplo de uso
alocacao = metodo_menor_custo(num_origens, num_destinos, cap_atendimento, demandas, matriz_custos)
print("Matriz de alocação inicial:")
print(alocacao)


Matriz de alocação inicial:
[[1000.    0.]
 [1300.  200.]
 [   0. 1200.]]


Interpretação da matriz de alocação inicial:

As linhas são as origens e as colunas são os destinos. Na primeira linha, a origem 1 mandou 1000 unidades para o destino 1, porque era o mais vantajoso. A origem 2 tem 1500 disponíveis. O menor custo para ele também é para o destino 1, então ele aloca 1300 unidades para atender o restante da demanda desse destino, sobrando 200, que são enviados para a segunda demanda.